In [22]:
import tensorflow as tf
import numpy as np
import os
from utils import *
from sklearn.preprocessing import MultiLabelBinarizer
import datetime
import pandas as pd

from apc import AveragePrecisionCalculator

In [ ]:
import hashlib
import itertools
import json
import os
from six.moves import urllib
import sys

In [16]:
tf.reset_default_graph()

In [17]:
# BUILD GRAPH
xdim = 1024+128
ydim = 3862
X = tf.placeholder(tf.float32, shape=[None, xdim])
Y = tf.placeholder(tf.float32, shape=[None, ydim])

h1 = lrelu(dense(X, xdim, 100, scope='h1'))
pred = dense(h1, 100, ydim, scope='h2')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=pred)
mloss = tf.reduce_mean(loss)

train = tf.train.AdamOptimizer().minimize(loss)

In [18]:
# INIT
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
# TRAIN
one_hot = MultiLabelBinarizer(classes=[i for i in range(3862)])
datadir = '/home/ec2-user/data/yt8m/train/'

# Iterate through train directory
for file in os.listdir(datadir)[:13]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

    onehot_labels = one_hot.fit_transform(labels)
    #onehot_labels = onehot_labels[:,:ydim]
    
    b = BatchIterator(onehot_labels, mean_rgb,mean_audio)
    # TRAIN
    for i in range(1001):
        batch = b.next_batch(32)
        if i % 500 == 0:
            loss = sess.run(mloss, feed_dict={X: batch[0], Y: batch[1]})
            print('step %d, loss %g' % (i, loss))
        sess.run(train, feed_dict={X: batch[0], Y: batch[1]})

    print('')
    
    
    # print('test accuracy %g' % sess.run(accuracy, 
    #                                     feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


train0111.tfrecord
step 0, loss 0.693111
step 500, loss 0.00420064
step 1000, loss 0.00291123
train2896.tfrecord
step 0, loss 0.124138
step 500, loss 0.0029563
step 1000, loss 0.00110892
train3607.tfrecord
step 0, loss 0.00669612
step 500, loss 0.0016076
step 1000, loss 0.000612419
train0580.tfrecord
step 0, loss 0.00533838
step 500, loss 0.00115437
step 1000, loss 0.000475373
train1864.tfrecord
step 0, loss 0.00320254
step 500, loss 0.000686389
step 1000, loss 0.000353773
train2950.tfrecord
step 0, loss 0.00748818
step 500, loss 0.00040733
step 1000, loss 0.000221918
train3012.tfrecord
step 0, loss 0.00278871
step 500, loss 0.000457206
step 1000, loss 9.11027e-05
train0667.tfrecord
step 0, loss 0.00378509
step 500, loss 0.000167778
step 1000, loss 5.15541e-05
train2072.tfrecord
step 0, loss 0.00336756
step 500, loss 0.000112916
step 1000, loss 4.79666e-05
train1805.tfrecord
step 0, loss 0.00274616
step 500, loss 8.8126e-05
step 1000, loss 2.73632e-05
train3749.tfrecord
step 0, loss 0.

In [20]:
#EVALUATE
datadir = '/home/ec2-user/data/yt8m/validate/'
APC = AveragePrecisionCalculator()
for file in os.listdir(datadir)[:10]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file    
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

    onehot_labels = one_hot.fit_transform(labels)
    #onehot_labels = onehot_labels[:,:869]
    
    b = BatchIterator(onehot_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, Y: b.y})

    ap = 0
    for i in range(p.shape[0]):
        ap+=APC.ap(p[i], b.y[i])

    print(ap/p.shape[0])

validate3182.tfrecord
0.6218261071217133
validate1831.tfrecord
0.6371668606909002
validate0701.tfrecord
0.6237080752484131
validate0780.tfrecord
0.5995237075496694
validate0052.tfrecord
0.6012274140282257
validate3043.tfrecord
0.6164242709291495
validate0955.tfrecord
0.6111037952060006
validate2094.tfrecord
0.6480942519489418
validate0523.tfrecord
0.6415045358514591


In [ ]:
#INFER
datadir = '/home/ec2-user/data/yt8m/test/'
outfile = 'out/'+ str(datetime.datetime.now())+'.csv'
with open(outfile, 'w') as f:
    pd.DataFrame(columns=['VideoIds','LabelConfidencePairs']).to_csv(f, index=False)
    
for file in os.listdir(datadir)[:3]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file

    print(file)
    
    vid_ids, _, mean_rgb, mean_audio = get_data_from_file(filename,has_labels=False)
    
    fake_labels = [[1] for _ in range(len(mean_rgb))]
    
    b = BatchIterator(fake_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x})
    
    top20 = p.argsort(axis=-1)[:,-20:][:,::-1]
    x_axis_index=np.tile(np.arange(len(p)), (top20.shape[1],1)).transpose() 
    probs = p[x_axis_index, top20]
    
    idx_probs = np.stack([top20, probs], axis=-1)
    idx_probs = [[(int(idx_probs[i,j,0]), idx_probs[i,j,1]) for j in range(idx_probs.shape[1])] for i in range(idx_probs.shape[0])]
    
    out = [str(idx_probs[0]).replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace(',', '') for i in range(len(idx_probs))]
    
    df = pd.DataFrame(data={'VideoId':vid_ids, 'LabelConfidencePairs':out})
    with open(outfile, 'a') as f:
        df.to_csv(f, header=False, index=False)
    
    
    

In [ ]:
datadir = '/home/ec2-user/data/yt8m/test/'
tempfile = '/home/ec2-user/data/yt8m/test_all/xx.tfrecord'
outfile = 'out/'+ str(datetime.datetime.now())+'.csv'

with open(outfile, 'w') as f:
    pd.DataFrame(columns=['VideoIds','LabelConfidencePairs']).to_csv(f, index=False)
    
    
download_plan = json.loads(urllib.request.urlopen(plan_url).read())
#files = [f for f in download_plan['files'].keys()]

ct = 0
start = datetime.datetime.now()
for f in files[:1]:
    ct+=1
    print('starting file {} of {}'.format(ct, len(files)))
    print('its been this much time: {}'.format(datetime.datetime.now()-start))
    print('{} % done'.format(int(ct/len(files)*100)))
    try:
        print('estimated time remaining {}'.format( (datetime.datetime.now()-start)/ ((ct-1)/(len(files)-(ct-1))) ))
    except:
        pass
    print(f)
    
    download_url = 'http://%s.data.yt8m.org/%s/%s' % (mirror, partition, f)
    download_file(download_url, tempfile)
    
    vid_ids, _, mean_rgb, mean_audio = get_data_from_file(filename,has_labels=False)
    
    fake_labels = [[1] for _ in range(len(mean_rgb))]
    b = BatchIterator(fake_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x})
    
    top20 = p.argsort(axis=-1)[:,-20:][:,::-1]
    x_axis_index=np.tile(np.arange(len(p)), (top20.shape[1],1)).transpose() 
    probs = p[x_axis_index, top20]
    idx_probs = np.stack([top20, probs], axis=-1)
    idx_probs = [[(int(idx_probs[i,j,0]), idx_probs[i,j,1]) for j in range(idx_probs.shape[1])] for i in range(idx_probs.shape[0])]
    out = [str(idx_probs[i]).replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace(',', '') for i in range(len(idx_probs))]
    df = pd.DataFrame(data={'VideoId':vid_ids, 'LabelConfidencePairs':out})
    
    with open(outfile, 'a') as f:
        df.to_csv(f, header=False, index=False)

starting file 1 of 3844
its been this much time: 0:00:00.000883
0 % done
starting file 2 of 3844
its been this much time: 0:00:00.001296
0 % done
estimated time remaining 0:00:05.956650
testAH.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 77880054 bytes.
starting file 3 of 3844
its been this much time: 0:00:01.464626
0 % done
estimated time remaining 0:46:54.057532
testSC.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 74401888 bytes.
starting file 4 of 3844
its been this much time: 0:00:02.859380
0 % done
estimated time remaining 1:01:01.306497
testAn.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 72222820 bytes.
starting file 5 of 3844
its been this much time: 0:00:04.211031
0 % done
estimated time remaining 1:07:22.873920
testLR.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 80375286 bytes.
starting file 6 of 3844
its been this much time: 0:00:05.706043
0 % don

starting file 40 of 3844
its been this much time: 0:01:04.713868
0 % done
estimated time remaining 1:45:13.775431
testyE.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 72015022 bytes.
starting file 41 of 3844
its been this much time: 0:01:06.280057
0 % done
estimated time remaining 1:45:03.258242
testDw.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 71310024 bytes.
starting file 42 of 3844
its been this much time: 0:01:09.075510
0 % done
estimated time remaining 1:46:47.198119
test5E.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 77401897 bytes.
starting file 43 of 3844
its been this much time: 0:01:11.315531
0 % done
estimated time remaining 1:47:35.776628
testgh.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78683905 bytes.
starting file 44 of 3844
its been this much time: 0:01:13.215816
0 % done
estimated time remaining 1:47:51.961551
testGI.tfrecord
Succesfull

Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 74079678 bytes.
starting file 79 of 3844
its been this much time: 0:02:30.677912
0 % done
estimated time remaining 2:01:15.051469
testqs.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 69674032 bytes.
starting file 80 of 3844
its been this much time: 0:02:33.039407
0 % done
estimated time remaining 2:01:33.599177
testut.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78770754 bytes.
starting file 81 of 3844
its been this much time: 0:02:34.836424
0 % done
estimated time remaining 2:01:25.065418
test2D.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 79269948 bytes.
starting file 82 of 3844
its been this much time: 0:02:36.751746
0 % done
estimated time remaining 2:01:22.194905
testWx.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 86351195 bytes.
starting file 83 of 3844
its been this much time: 0:02:

starting file 117 of 3844
its been this much time: 0:03:51.920061
0 % done
estimated time remaining 2:04:13.439121
test7L.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 71162533 bytes.
starting file 118 of 3844
its been this much time: 0:03:53.886322
0 % done
estimated time remaining 2:04:10.387645
testSw.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 68202816 bytes.
starting file 119 of 3844
its been this much time: 0:03:55.864050
0 % done
estimated time remaining 2:04:07.715763
testgu.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 83713428 bytes.
starting file 120 of 3844
its been this much time: 0:03:57.813738
0 % done
estimated time remaining 2:04:04.178169
testKz.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 75983152 bytes.
starting file 121 of 3844
its been this much time: 0:03:59.809033
0 % done
estimated time remaining 2:04:02.082409
testyW.tfrecord
Succe

starting file 155 of 3844
its been this much time: 0:05:08.133705
0 % done
estimated time remaining 2:03:03.210180
test3p.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 79575978 bytes.
starting file 156 of 3844
its been this much time: 0:05:10.052779
0 % done
estimated time remaining 2:02:59.262709
testuG.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 71331851 bytes.
starting file 157 of 3844
its been this much time: 0:05:11.869465
0 % done
estimated time remaining 2:02:52.920615
testhk.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 74261094 bytes.
starting file 158 of 3844
its been this much time: 0:05:14.003516
0 % done
estimated time remaining 2:02:54.088676
testZt.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 81326100 bytes.
starting file 159 of 3844
its been this much time: 0:05:16.097740
0 % done
estimated time remaining 2:02:54.286696
testRZ.tfrecord
Succe

...

starting file 2303 of 3844
its been this much time: 5:50:00.002704
0 % done
estimated time remaining 3:54:26.900349
testcg.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 73741746 bytes.
starting file 2304 of 3844
its been this much time: 5:50:09.676555
0 % done
estimated time remaining 3:54:18.146758
testld.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 72825240 bytes.
starting file 2305 of 3844
its been this much time: 5:50:19.243186
0 % done
estimated time remaining 3:54:09.320732
testr5.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78088371 bytes.
starting file 2306 of 3844
its been this much time: 5:50:29.495715
0 % done
estimated time remaining 3:54:00.951963
test8b.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78464854 bytes.
starting file 2307 of 3844
its been this much time: 5:50:39.818409
0 % done
estimated time remaining 3:53:52.628414
testfw.tfrecord


starting file 2341 of 3844
its been this much time: 5:56:16.995986
0 % done
estimated time remaining 3:48:59.744625
testdc.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 65948802 bytes.
starting file 2342 of 3844
its been this much time: 5:56:25.631424
0 % done
estimated time remaining 3:48:50.288091
testY0.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 72895638 bytes.
starting file 2343 of 3844
its been this much time: 5:56:35.197581
0 % done
estimated time remaining 3:48:41.429219
testy1.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 74317379 bytes.
starting file 2344 of 3844
its been this much time: 5:56:45.092853
0 % done
estimated time remaining 3:48:32.780512
test0h.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 73866158 bytes.
starting file 2345 of 3844
its been this much time: 5:56:54.633232
0 % done
estimated time remaining 3:48:23.903684
testMr.tfrecord


starting file 2379 of 3844
its been this much time: 6:02:35.240275
0 % done
estimated time remaining 3:43:31.767296
test9b.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 76892310 bytes.
starting file 2380 of 3844
its been this much time: 6:02:45.327026
0 % done
estimated time remaining 3:43:23.196530
testW6.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 68683684 bytes.
starting file 2381 of 3844
its been this much time: 6:02:54.340895
0 % done
estimated time remaining 3:43:13.964491
test1Q.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 76583100 bytes.
starting file 2382 of 3844
its been this much time: 6:03:04.388877
0 % done
estimated time remaining 3:43:05.368041
testPd.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 79759602 bytes.
starting file 2383 of 3844
its been this much time: 6:03:14.850266
0 % done
estimated time remaining 3:42:57.024125
testsW.tfrecord


starting file 2417 of 3844
its been this much time: 6:08:45.408180
0 % done
estimated time remaining 3:37:57.435149
testhS.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 73108840 bytes.
starting file 2418 of 3844
its been this much time: 6:08:54.996065
0 % done
estimated time remaining 3:37:48.531148
test8E.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 73982832 bytes.
starting file 2419 of 3844
its been this much time: 6:09:04.697640
0 % done
estimated time remaining 3:37:39.693646
testKM.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78893802 bytes.
starting file 2420 of 3844
its been this much time: 6:09:15.047335
0 % done
estimated time remaining 3:37:31.237217
testYX.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 68973070 bytes.
starting file 2421 of 3844
its been this much time: 6:09:24.092243
0 % done
estimated time remaining 3:37:22.011476
testVR.tfrecord


starting file 2455 of 3844
its been this much time: 6:15:03.058096
0 % done
estimated time remaining 3:32:26.234769
testCS.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 78187892 bytes.
starting file 2456 of 3844
its been this much time: 6:15:13.333463
0 % done
estimated time remaining 3:32:17.687996
testtq.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 76954386 bytes.
starting file 2457 of 3844
its been this much time: 6:15:23.587984
0 % done
estimated time remaining 3:32:09.131874
testfv.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 72831218 bytes.
starting file 2458 of 3844
its been this much time: 6:15:33.131813
0 % done
estimated time remaining 3:32:00.171391
testHO.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 81986310 bytes.
starting file 2459 of 3844
its been this much time: 6:15:43.843849
0 % done
estimated time remaining 3:31:51.869555
tests0.tfrecord


In [ ]:
p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, Y: b.y})

predicted = one_hot.inverse_transform(np.concatenate([np.array(p>.5, dtype=int), np.zeros([292,3862-869])], axis=1))
actual = one_hot.inverse_transform(np.concatenate([b.y, np.zeros([292,3862-869])], axis=1))




In [ ]:
correct=0

for i in range(len(actual)):
    if actual[i]==predicted[i]:
        correct+=1
print(correct/len(actual))

In [ ]:
actual[0]==predicted[0]

In [ ]:
actual

In [ ]:
predicted

In [ ]:
p.shape

In [ ]:
one_hot.inverse_transform(np.concatenate([batch[1], np.zeros([32,3862-869])], axis=1))

In [ ]:
one_hot.classes[:869][batch[1][0]>.5]

In [ ]:
one_hot.classes

In [ ]:
np.zeros(2)[[True,False]]

In [ ]:
datadir = '/home/ec2-user/data/yt8m/test/'

In [ ]:
file=os.listdir(datadir)[5]
filename = datadir + file
for example in tf.python_io.tf_record_iterator(filename):
    tf_example = tf.train.Example.FromString(example)
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

In [ ]:
def get_data_from_file(filename, has_labels=True):
    vid_ids, labels, mean_rgb, mean_audio = [], [], [], []

    for example in tf.python_io.tf_record_iterator(filename):
        tf_example = tf.train.Example.FromString(example)

        vid_ids.append(tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
        labels.append(tf_example.features.feature['labels'].int64_list.value)
        mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
        mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)
    return vid_ids, labels, mean_rgb, mean_audio

In [ ]:
labels[-1]

In [ ]:
tf_example

In [ ]:
p = sess.run(tf.sigmoid(pred),
        feed_dict={X: batch[0], Y: batch[1]})

In [ ]:
np.sum(p>.5, axis=1)

In [ ]:
one_hot.classes_[p[6]>.5]

In [ ]:
yy = batch[1][6]

In [ ]:
one_hot.classes_[yy>.5]

In [ ]:
yy

In [ ]:
import apc_file

In [ ]:
apcc = apc_file.AveragePrecisionCalculator()

In [ ]:
apcc.ap(p[2], b.y[2])

In [ ]:
p